In [ ]:
!pip install turicreate

In [3]:
import numpy as np
import pandas as pd
import gc 
pd.set_option('display.max_columns', 50)
import warnings
import turicreate as tc
tc.config.set_num_gpus(1)
warnings.filterwarnings("ignore")

In [4]:
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import random
import datetime
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from sklearn.preprocessing import LabelEncoder

In [6]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/JanathaHack/RecommendationSystems/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/JanathaHack/RecommendationSystems/test.csv')
combine_set=pd.concat([train,test], ignore_index=True)
challenge = pd.read_csv("/content/drive/My Drive/Colab Notebooks/JanathaHack/RecommendationSystems/challenge_data.csv")
le =LabelEncoder()
challenge = challenge[["challenge","programming_language",
                       "challenge_series_ID","total_submissions",
                       "category_id","publish_date"]]
challenge["challenge_series_ID"] = le.fit_transform(challenge["challenge_series_ID"].astype(str))
challenge["total_submissions"] = challenge["total_submissions"].fillna(challenge["total_submissions"].mean()).astype(int)
challenge["category_id"] = challenge["category_id"].fillna(challenge["category_id"].mean()).astype(int)
challenge['publish_date'] = pd.DatetimeIndex(challenge['publish_date']).year


mer_train = pd.merge(train,challenge,on=["challenge"],how="left")
mer_test = pd.merge(test,challenge,on=["challenge"],how="left")
mer_df = mer_train.append(mer_test).reset_index(drop=True)
user_df = mer_df.groupby("user_id")["challenge_series_ID"].agg(lambda x: pd.Series.mode(x)[0]).to_frame()
user_df = user_df.reset_index()
user_df = tc.SFrame(user_df)
challenge_df = tc.SFrame(challenge)

In [7]:
combine_set_tc=tc.SFrame(combine_set)
m=tc.item_similarity_recommender.create(combine_set_tc, user_id='user_id',
                                                            item_id='challenge',
                                                            target='challenge_sequence',
                                                            user_data=user_df,
                                                            item_data = challenge_df,
                                                            similarity_type = "cosine",
                                                            only_top_k=200,
                                                            )

Warning: Ignoring columns user_sequence;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 1301236 observations with 109264 users and 5606 items.

Data prepared in: 1.32783s

...

| 2.56s                               | 100              | 5606            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.76601s

In [8]:
results = m.recommend(test["user_id"].unique().tolist(),k=3)["challenge"]
results[:10]

recommendations finished on 1000/39732 queries. users per second: 8300

recommendations finished on 2000/39732 queries. users per second: 8314.11

recommendations finished on 3000/39732 queries. users per second: 8358.13

recommendations finished on 4000/39732 queries. users per second: 8333.84

recommendations finished on 5000/39732 queries. users per second: 8287.43

...

recommendations finished on 36000/39732 queries. users per second: 8407.28

recommendations finished on 37000/39732 queries. users per second: 8413.45

recommendations finished on 38000/39732 queries. users per second: 8427.59

recommendations finished on 39000/39732 queries. users per second: 8471.64

dtype: str
Rows: 10
['CI23848', 'CI23691', 'CI25126', 'CI24530', 'CI23848', 'CI25126', 'CI26953', 'CI26959', 'CI26958', 'CI23648']

In [9]:
submission_sample =pd.read_csv('/content/drive/My Drive/Colab Notebooks/JanathaHack/RecommendationSystems/sample_sub.csv')

submission_sample["challenge"] = np.array(results).reshape(-1,1)

submission_sample.to_csv("submit_target.csv",index=False)